# Modeling Stat Interactions based on the Call of Cthulhu board game

In the *Call of Cthulhu* board game, player character stats can be grouped into "characteristics", "skills", "attributes", and "occupations".

## Characteristics

Characteristics can be further distinguished by whether they are *direct* or *indirect*. Direct characteristics are determined by player dice rolls or choice, while indirect characteristics are calculated once direct ones have been determined.

There are ten direct characteristics in the game:
1. Age
2. Appearance
3. Constitution
4. Dexterity
5. Education
6. Intelligence
7. Luck
8. Power
9. Size
10. Strength

Of the ten characteristics, Age (1) and Luck (7) do not directly factor into skill allotment. Age has some indirect 

There are three indirect characteristics in the game:
1. Damage Bonus and Build
2. Hit Points
3. Movement Rate

In [109]:
import random
import numpy as np

# array of tuples of the form (stat_min, stat_max). The values are taken from the
# Characteristics section of the player handbook
CHARACTERISTICS = {
    'AGE': (15, 90), # "...choose any age between 15 and 90..."
    'APPEARANCE': (15, 90), # "...Roll 3D6 and Multiply by 5..."
    'CONSTITUTION': (15, 90), # "...Roll 3D6 and Multiply by 5..."
    'DEXTERITY': (15, 90), # "...Roll 3D6 and Multiply by 5..."
    'EDUCATION': (40, 90), # "...Roll 2D6 + 6 and Multiply by 5..."
    'INTELLIGENCE': (40, 90), # "...Roll 2D6 + 6 and Multiply by 5..."
    'LUCK': (40, 90), # "...Roll 2D6 + 6 and Multiply by 5..."
    'POWER': (15, 90), # "...Roll 3D6 and Multiply by 5..."
    'SIZE': (40, 90), # "...Roll 2D6 + 6 and Multiply by 5..."
    'STRENGTH': (15, 90) # "...Roll 3D6 and Multiply by 5..."
}

PI_MULTIPLIER = 2.

def indexOf(key, dictionary):
    return list(dictionary.keys()).index(key)

assert indexOf('INTELLIGENCE', CHARACTERISTICS) == 5

class Character():
    def __init__(self, characteristics=None, age=None):
        # public
        self.HP = 0
        self.MOV = 0
        self.characteristics = characteristics
        # self.occupation = Occupation()
        # self.skills = []
        # private
        
        # initialization
        if characteristics is None:
            self.roll_characteristics()
            
        if age is not None:
            self['AGE'] = age
            
        self.apply_age_modifier()
            
    def __getitem__(self, key):
        return self.get_characteristic_val(key)
    
    def __setitem__(self, key, value):
        self.characteristics[indexOf(key, CHARACTERISTICS)] = value
    
    def modify_characteristic(self, characteristic, func):
        self.characteristics[indexOf(characteristic, CHARACTERISTICS)] = max(0, func(self.get_characteristic_val(characteristic)))
    
    def make_distributed_modification(self, characteristics, func, starting_mod):
        remaining_mod = starting_mod
        for c in characteristics:
            m = np.random.randint(0, remaining_mod)
            self.modify_characteristic(c, lambda cv: max(0, func(m, cv)))
            remaining_mod = remaining_mod - m
    
    def set_characteristics(self, new_characteristics):
        self.characteristics = new_characteristics
        self.apply_age_modifier()
        self.set_base_HP()
        self.set_base_movement()
    
    def set_base_HP(self):
        self.HP = (self['CONSTITUTION'] + self['SIZE']) // 10
    
    def set_base_movement(self):
        if self['DEXTERITY'] < self['SIZE'] and self['STRENGTH'] < self['SIZE']:
            self.MOV = 7
        elif self['DEXTERITY'] > self['SIZE'] or self['STRENGTH'] > self['SIZE']:
            self.MOV = 8
        elif self['DEXTERITY'] > self['SIZE'] and self['STRENGTH'] > self['SIZE']:
            self.MOV = 9
        
        if self['AGE'] < 40:
            pass
        elif self['AGE'] <= 49:
            self.MOV = self.MOV - 1
        elif self['AGE'] <= 59:
            self.MOV = self.MOV - 2
        elif self['AGE'] <= 69:
            self.MOV = self.MOV - 3
        elif self['AGE'] <= 79:
            self.MOV = self.MOV - 4
        elif self['AGE'] <= 90:
            self.MOVE = self.MOV - 5
            
    def get_personal_interest_points(self):
        return self.get_characteristic_val('INTELLIGENCE') * PI_MULTIPLIER

    def get_characteristic_val(self, characteristic):
        return self.characteristics[indexOf(characteristic, CHARACTERISTICS)]
    
    def roll_characteristics(self):
        new_characteristics = np.empty(len(CHARACTERISTICS))
        for i, c in enumerate(CHARACTERISTICS):
            char_range = CHARACTERISTICS[c]
            new_characteristics[i] = np.random.randint(char_range[0], char_range[1])
            
        self.set_characteristics(new_characteristics)
            
    def apply_age_modifier(self):
        age = self.get_characteristic_val('AGE')
        if age <= 19:
            self.modify_characteristic('STRENGTH', lambda s: s - 5)
            self.modify_characteristic('SIZE', lambda s: s - 5)
            self.modify_characteristic('EDUCATION', lambda e: e - 5)
            luck_ranges = CHARACTERISTICS['LUCK']
            self.modify_characteristic('LUCK', lambda l: np.max([l, np.random.randint(luck_ranges[0], luck_ranges[1])]))
        
        elif age <= 39:
            self._improve_education(1)
        
        elif age <= 49:
            self._improve_education(2)
            self.make_distributed_modification(('STRENGTH', 'CONSTITUTION', 'DEXTERITY'), lambda m, c: c - m, 5)
            self.modify_characteristic('APPEARANCE', lambda a: a - 5)
            
        elif age <= 59:
            self._improve_education(3)
            self.make_distributed_modification(('STRENGTH', 'CONSTITUTION', 'DEXTERITY'), lambda m, c: c - m, 10)
            self.modify_characteristic('APPEARANCE', lambda a: a - 10)
        
        elif age <= 69:
            self._improve_education(4)
            self.make_distributed_modification(('STRENGTH', 'CONSTITUTION', 'DEXTERITY'), lambda m, c: c - m, 20)
            self.modify_characteristic('APPEARANCE', lambda a: a - 15)
        
        elif age <= 79:
            self._improve_education(4)
            self.make_distributed_modification(('STRENGTH', 'CONSTITUTION', 'DEXTERITY'), lambda m, c: c - m, 40)
            self.modify_characteristic('APPEARANCE', lambda a: a - 20)
            
        elif age <= 90:
            self._improve_education(4)
            self.make_distributed_modification(('STRENGTH', 'CONSTITUTION', 'DEXTERITY'), lambda m, c: c - m, 80)
            self.modify_characteristic('APPEARANCE', lambda a: a - 25)
        
    def _improve_education(self, amt_checks):
        improvement_checks = np.random.randint(1, 100, size=amt_checks)
        base_edu = self.get_characteristic_val('EDUCATION')
        new_edu = base_edu
        for ic in improvement_checks:
            if ic > base_edu:
                new_edu = new_edu + np.random.randint(1, 10)
        
        self.modify_characteristic('EDUCATION', lambda e: min(new_edu, 99))

In [113]:
character = Character()
character.characteristics[indexOf('INTELLIGENCE', CHARACTERISTICS)] = 24
assert np.isclose(
    24 * 2.,
    character.get_personal_interest_points()
)

# lengthy calculation
# it passed already, but uncomment to check again
# for i in range(10000):
#     character = Character()
#     assert np.all(character.characteristics >= 0)

## Occupations
Occupations are like traditional RPG classes in a loose sense. They don't yield unique abilities, powers, or perks, but instead dictate what the skills the character is good at. There are too many to list here from the player handbook, so we'll just discuss them abstractly here.

Mathematically, we can see occupations as tuples of the form $(s, \gamma)$, with $s$ being a set of skills, $|s| = 8$, and $\gamma = \{ x_i \in \mathbb{R} | 1 \leq i \leq 10 \}$ to be the *skill transform*, which, when applied to the vector of character characteristics, yields the amount of skill points the occupation grants the character.

In [56]:
class Occupation():
    def __init__(self, skills, credit_rating, skill_transform):
        self._skills = skills
        self._CR = credit_rating
        
    def get_skills(self):
        return self._skills
    
    def get_credit_rating(self):
        return self._CR
    
    def occupation_skill_points(self, characteristics):
        return self.skill_function(characteristics)

In [57]:
class Skill():
    def __init__(self, base_value):
        self._base_value = base_value
        
    def get_base_value(self):
        return self._base_value

AttributeError: 'Character' object has no attribute '_get_characteristic_val'